In [1]:
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:0000:0100:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845553 sha256=382cfa9b3100052e08c3c2b15cd1b6b854474fe66987bad257ffd0f50c450888
  Stored in directory: /home/dev/.cache/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/19 12:14:21 WARN Utils: Your hostname, ANDERSON resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/19 12:14:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/19 12:14:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json('../../curso2/datalake/twitter_datascience')

In [4]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{19, 32, 2025-01...|{[{2025-01-14T21:...|{1234567890abcdef}|  2025-01-14|
|[{37, 42, 2025-01...|{[{2025-01-14T05:...|              null|  2025-01-14|
|[{77, 90, 2025-01...|{[{2025-01-16T18:...|              null|  2025-01-16|
|[{55, 51, 2025-01...|{[{2025-01-12T23:...|              null|  2025-01-12|
|[{19, 87, 2025-01...|{[{2025-01-17T10:...|              null|  2025-01-17|
|[{43, 86, 2025-01...|{[{2025-01-13T20:...|              null|  2025-01-13|
|[{2, 66, 2025-01-...|{[{2025-01-15T03:...|              null|  2025-01-15|
|[{58, 46, 2025-01...|{[{2025-01-11T20:...|              null|  2025-01-11|
|[{17, 2, 2025-01-...|{[{2025-01-10T08:...|              null|  2025-01-10|
+--------------------+--------------------+------------------+------------+



In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [6]:
from pyspark.sql import functions as f

In [7]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{19, 32, 2025-01-...|
|{93, 35, 2025-01-...|
|{81, 47, 2025-01-...|
|{88, 44, 2025-01-...|
|{62, 54, 2025-01-...|
|{51, 45, 2025-01-...|
|{37, 73, 2025-01-...|
|{20, 32, 2025-01-...|
|{81, 25, 2025-01-...|
|{79, 36, 2025-01-...|
|{37, 42, 2025-01-...|
|{13, 56, 2025-01-...|
|{87, 1, 2025-01-1...|
|{82, 17, 2025-01-...|
|{3, 11, 2025-01-1...|
|{26, 47, 2025-01-...|
|{49, 0, 2025-01-1...|
|{88, 16, 2025-01-...|
|{39, 0, 2025-01-1...|
|{65, 51, 2025-01-...|
+--------------------+
only showing top 20 rows



In [9]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [10]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [11]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       19|             32|2025-01-14T07:14:...| 15|        18|         30|         21|           56|Tweet fictício cr...|
|       93|             35|2025-01-14T19:35:...| 11|        88|         48|          0|           19|Um terceiro tweet...|
|       81|             47|2025-01-14T20:30:...| 82|         2|         62|         17|           16|Tweet fictício cr...|
|       88|             44|2025-01-14T06:11:...| 27|        72|         66|         19|           31|Um terceiro tweet...|
|       62|             54|2025-01-14T09:23:...| 68|        70|         88|         93|           17|Um terceiro tweet...|
+---------+-----

In [12]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [14]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [15]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [17]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2025-01-14T21:36:...|  4|User 1|   user1|
|2025-01-14T02:09:...| 88|User 2|   user2|
|2025-01-14T13:15:...| 29|User 3|   user3|
|2025-01-14T10:14:...| 53|User 4|   user4|
|2025-01-14T03:12:...|  1|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [18]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')

In [19]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')